In [ ]:
%aiida
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
    
    
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm import load_node, Code, Computer
from aiida.common.exceptions import NotExistent, MultipleObjectsError

In [ ]:
## -------------------------------------------------------
## List computers
qb = QueryBuilder()
qb.append(Computer)

for e in qb.all():
    print(e)
## -------------------------------------------------------

In [ ]:
## -------------------------------------------------------
## List  codes
qb = QueryBuilder()
qb.append(Code)

for e in qb.all():
    print(e)
## -------------------------------------------------------

In [ ]:
# 1. Select computer

computer_name = "daint-s746"
computer = Computer.get(computer_name)

In [ ]:
# Quantum espresso

exe_dir = "/users/keimre/soft/qe-6.3-gpu-pgi/bin/"
exe_list = ["pw", "pp", "ph", "projwfc", "matdyn", "q2r"]
version_id = "6.3-custom2"

prepend_text  = "### code prepend_text start ###\n"
#prepend_text += "module load daint-gpu\n"
prepend_text += "### code prepend_text end ###\n"

for exe in exe_list:
    code_label = exe+"-"+version_id
    
    code_full_name = code_label + "@" + computer_name

    try:
        Code.get_from_string(code_full_name)
    except (NotExistent, MultipleObjectsError):            
        code = Code(remote_computer_exec=(computer, exe_dir + exe + ".x"))
        code.label = code_label
        code.description = "Quantum Espresso %s %s.x" % (version_id, exe)
        code.set_input_plugin_name("quantumespresso.{}".format(exe))
        code.set_prepend_text(prepend_text)
        code.set_append_text("")
        code.store()
        ! verdi code show "{code_full_name}"
    else:
        print("Code '{}' already existent".format(code_full_name))  

# Remove codes

In [ ]:
## -------------------------------------------------------
## List  codes
qb = QueryBuilder()
qb.append(Code)

for e in qb.all():
    print(e)
## -------------------------------------------------------

In [ ]:
! verdi code delete 416